In [65]:
import pandas as pd
import math
import os
from tqdm.notebook import tqdm
import networkx as nx
import numpy as np
import scipy.sparse as sp
# from nltk.tokenize.punkt import PunktSentenceTokenizer
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from deep_translator import GoogleTranslator

In [66]:
def summarize_textrank(sentences):
    # sentence_tokenizer = PunktSentenceTokenizer()
    # sentences = sentence_tokenizer.tokenize(document)

    bow_matrix = CountVectorizer().fit_transform(sentences)
    normalized = TfidfTransformer().fit_transform(bow_matrix)

    similarity_graph = normalized * normalized.T
    sparse_matrix = sp.csr_matrix(similarity_graph)
    nx_graph = nx.Graph(sparse_matrix)
    scores = nx.pagerank(nx_graph)
    sentence_array = sorted(((scores[i], s) for i, s in enumerate(sentences)), reverse=True)
    
    sentence_array = np.asarray(sentence_array)
    
    fmax = float(sentence_array[0][0])
    fmin = float(sentence_array[len(sentence_array) - 1][0])
    
    temp_array = []
    for i in range(0, len(sentence_array)):
        if fmax - fmin == 0:
            temp_array.append(0)
        else:
            temp_array.append((float(sentence_array[i][0]) - fmin) / (fmax - fmin))

    threshold = (sum(temp_array) / len(temp_array)) + 0.2
    
    sentence_list = []

    for i in range(0, len(temp_array)):
        if temp_array[i] > threshold:
            sentence_list.append(sentence_array[i][1])

    seq_list = []
    for sentence in sentences:
        if sentence in sentence_list:
            seq_list.append(sentence)
    return seq_list

In [67]:
def split_long_sentence(sentence, max_length=3000):
    parts = []
    for i in range(0, len(sentence), max_length):
        part = sentence[i:i + max_length]
        parts.append(part)

    return parts

def convertToHindi(sentences):
    hindi = []
    for sentence_array in sentences:
        sentence = ''
        hindiText=''
        for i in range(len(sentence_array)):
            sentence = sentence + sentence_array[i]
        if len(sentence)>2000:
            small_sentences = split_long_sentence(sentence, max_length=2000)
            hindiPart = ''
            for i in range(len(small_sentences)):
                current_translate = GoogleTranslator(source='en', target='hi').translate(small_sentences[i])
                hindiPart = hindiPart + current_translate
        else:
            hindiPart = GoogleTranslator(source='en', target='hi').translate(sentence)
        hindiText=hindiText + hindiPart
        hindi.append(hindiText)
    return hindi

In [68]:
input_path="./data/summary/"
output_path="./data/summary_results/"
if os.path.exists(output_path) == False:
    os.mkdir(output_path)

data = pd.read_csv(f'{input_path}data.csv')
data = data.head(2)
sentences = data['sentences'].apply(eval)
sentences_english = data['sentences_english'].apply(eval)

summary=[]
summary_english=[]
print(summarize_textrank(sentences[0]))
for i in tqdm(range(len(sentences))):
    summary.append(summarize_textrank( sentences[i]) ) 
    summary_english.append([convertToHindi(summarize_textrank( sentences_english[i]) )])

data['summary'] = summary
data['summary_english'] = summary_english

data.to_csv(f'{output_path}textRank.csv', index=False)


['अभियोजन कथानक संक्षेप में इस प्रकार है कि वादी मुकदमा लक्ष्मीनरायन ने थाना शाहगंज <नाम> दिनाक 09.08.2020 को इस आशय की तहरीर दी किउसका स्कूटर मेस्ट्रो रंग स्काई ब्लू नम्बर यूपी.80 डी.डी.2730 दिनांक 22.07.2020 कोरात्रि 9.30 बजे उसके निवास <नाम> ताला लगा खड़ी थी, तभी समय करीब 10 बजेउसने <नाम> तो उसके उक्त स्कूटर को अज्ञात चोर चोरी करके ले गया है', 'अभियुक्त के विद्वान अधिवक्ता द्वारा जमानत प्रार्थनापत्र/ शपथपत्र मेंकथन किया गया कि आवेदक/ अभियुक्त ने कोई अपराध कारित नहीं किया है तथाउसे झूंठा फंसाया गया है', 'आवेदक / अभियुक्त से अपराध से संबंधित कोई बरामदगीनहीं है', 'आवेदक / अभियुक्त का उक्त अपराध में न्यायालय में यह प्रथम जमानत्रार्थनापत्र है, इसके अलावा आवेदक / अभियुक्त का किसी अन्य न्यायालय में कोईजमानत प्रार्थनापत्र विचाराधीन नहीं है']


  0%|          | 0/2 [00:00<?, ?it/s]